# install rdkit via pip

!pip install rdkit

# imports

In [1]:
import os

import mlx.core as mx
import scipy as sp
import numpy as np
import pandas as pd
from mlx_graphs.data import GraphData
from mlx_graphs.datasets.dataset import Dataset
from mlx_graphs.datasets.utils import download
from mlx_graphs.utils.transformations import to_sparse_adjacency_matrix
from typing import Tuple
from typing import Optional

from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors


# get the ESOL source file

In [2]:
download('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv', path="ESOL.csv")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96.7k/96.7k [00:00<00:00, 321kB/s]


'ESOL.csv'

# retreive SMILES and target LogS columns only

In [3]:
data = pd.read_csv('ESOL.csv')
data = data[['smiles','measured log solubility in mols per litre']]
data.columns = ['Smiles','LogS']
data.head()

,Smiles,LogS
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.77
1,Cc1occc1C(=O)Nc2ccccc2,-3.30
2,CC(C)=CCCC(C)=CC(=O),-2.06
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.87
4,c1ccsc1,-1.33


In [4]:
# 



In [5]:

# my c++ official RDkit code to get Atom basic features 
def _atomFeatures(atom: Chem.Atom,
                 ) -> np.array:
    return rdMolDescriptors.GetAtomFeatures(atom.GetOwningMol(), atom.GetIdx())

# AdjacencyMatrix extractoin and formating
def _compute_adjacency(
    molecule: Chem.Mol,
    dtype: np.dtype = np.int32,
) -> Tuple[np.ndarray, np.ndarray]:
    'Computes adjacency matrix from an RDKit molecule object.'

    adjacency = Chem.GetAdjacencyMatrix(molecule)

    return adjacency.astype(dtype)

# Get Edge_features
def generate_bond_features(
    mol: Chem.Mol)-> Tuple[np.ndarray, np.ndarray]:
    # Dictionaries for mapping bond types and stereochemistry to integers
    bond_type_dict = {'SINGLE': 1, 'DOUBLE': 2, 'TRIPLE': 3, 'AROMATIC': 4}
    bond_stereo_dict = {'STEREONONE': 0,'STEREOANY': 1, 'STEREOE': 2, 'STEREOZ': 3}
    
    # Calculate rotatable bonds
    rotbonds = Lipinski._RotatableBonds(mol)
    
    # Initialize a list to store bond features
    bond_features = []
    
    for bond in mol.GetBonds():
        # Get the owning molecule (not necessary if `mol` is already given)
        mol = bond.GetOwningMol()
        
        # Get sorted atom indices for the bond
        atom_indices = tuple(sorted([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]))
        
        # Determine if the bond is rotatable
        is_rotatable = atom_indices in rotbonds
        
        # Get the bond's features
        bond_stereo_feature = bond_stereo_dict[bond.GetStereo().name]
        bond_type_feature = bond_type_dict[bond.GetBondType().name]
        is_conjugated = bond.GetIsConjugated()
        
        # Append the features as a tuple to the bond_features list
        bond_features.append((bond_stereo_feature, bond_type_feature, is_conjugated, is_rotatable))
    
    return bond_features

In [6]:
# generate the dataset

In [7]:
dataset = []
for i,row in data.iterrows():
    try:
        # Get an RDkit Molecule object from Smiles
        mol = Chem.MolFromSmiles(row.Smiles.strip())
        # Get the edge_indexes (we will overwrite the edge_features)
        edge_index, edge_features = to_sparse_adjacency_matrix(mx.array(_compute_adjacency(mol)))
        # Get Node features
        atf = np.zeros((mol.GetNumAtoms(),49))
        for i, atom in enumerate(mol.GetAtoms()):
            atf[i,:] = _atomFeatures(atom)
        node_features  = mx.array(atf)
        # Get Edge features
        edge_features = mx.array(generate_bond_features(mol))
        # Get the target : "LogS"
        label =  mx.ones((1, 1))*row.LogS
        # append the list of GraphData objects
        dataset.append(
            GraphData(
                
                edge_index=edge_index,
                node_features=node_features,
                edge_features=edge_features,
                graph_labels=label, 
                        )
                )
    except:
        # the "C" is a single atom molecule so it is expected to be an exception in this process!
        print(row)

Smiles      C
LogS     -0.9
Name: 934, dtype: object


In [8]:
label

array([[-4.522]], dtype=float32)

In [9]:
row.LogS

-4.522

In [10]:
dataset[0]

GraphData(
	edge_index(shape=(2, 68), uint32)
	node_features(shape=(32, 49), float32)
	edge_features(shape=(34, 4), int32)
	graph_labels(shape=(1, 1), float32))

In [11]:
from mlx_graphs.loaders import Dataloader

# split is an example not a real value of CV also we need to add the randomization!!!
train_dataset = dataset[:1000]
test_dataset = dataset[1000:]


BATCH_SIZE = 64

train_loader = Dataloader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = Dataloader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

for batch in train_loader:
    print(f"\nGraph batch of size {len(batch)}")
    print(batch)
    print(batch.batch_indices)
    break


Graph batch of size 64
GraphDataBatch(
	edge_index(shape=(2, 1796), int64)
	node_features(shape=(869, 49), float32)
	edge_features(shape=(898, 4), int32)
	graph_labels(shape=(64, 1), float32))
array([0, 0, 0, ..., 63, 63, 63], dtype=int64)


In [12]:
import mlx.nn as nn
from mlx_graphs.nn import GINConv, global_mean_pool, Linear
import time


class GINE(nn.Module):
    def __init__(self, in_dim_edge, in_dim_node, hidden_dim, out_dim, dropout=0.1):
        super(GINE, self).__init__()

        self.conv1 = GINConv(
            mlp=nn.Sequential(
            nn.Linear(in_dim_node, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            ), 
            #node_features_dim=in_dim_node,
            #edge_features_dim=in_dim_edge,
        )
        self.conv2 = GINConv(
            mlp=nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            ), 
            #node_features_dim=hidden_dim,
            #edge_features_dim=hidden_dim,
        )
        self.conv3 = GINConv(
            mlp=nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            ), 
            #node_features_dim=hidden_dim,
            #edge_features_dim=hidden_dim,
        )
        self.linear = Linear(hidden_dim, out_dim)

        self.dropout = nn.Dropout(p=dropout)

    def __call__(self, edge_index, node_features, batch_indices):
        h = nn.relu(self.conv1(edge_index, node_features))
        h = nn.relu(self.conv2(edge_index, h))
        h = self.conv3(edge_index, h)
        
        h = global_mean_pool(h, batch_indices)

        h = self.dropout(h)
        h = self.linear(h)
        
        return h

In [13]:
def loss_fn(y_hat, y, parameters=None):
    return mx.mean(nn.losses.mse_loss(y_hat, y))

def forward_fn(model, graph, labels):
    y_hat = model(graph.edge_index, graph.node_features, graph.batch_indices)
    loss = loss_fn(y_hat, labels, model.parameters())
    return loss, y_hat

In [14]:
device = mx.gpu # or mx.cpu
mx.set_default_device(device)

In [15]:
def train(train_loader):
    loss_sum = 0.0
    for graph in train_loader:
        
        (loss, y_hat), grads = loss_and_grad_fn(
            model=model,
            graph=graph,
            labels=graph.graph_labels,
        )
        optimizer.update(model, grads)
        mx.eval(model.parameters(), optimizer.state)
        loss_sum += loss.item()
    return loss_sum / len(train_loader.dataset)

def test(test_loader):
    mse= 0.0
    for graph in test_loader:
        y_hat = model(graph.edge_index, graph.node_features, graph.batch_indices)
        mse += mx.square(y_hat - graph.graph_labels).sum().item()
    
    return mse / len(test_loader.dataset)

def epoch():
    loss = train(train_loader)
    train_mse = test(train_loader)
    test_mse = test(test_loader)
    return loss, train_mse, test_mse

In [18]:
import mlx.optimizers as optim
mx.random.seed(42)

model = GINE(
    in_dim_node=dataset[0].node_features.shape[1],
    in_dim_edge=dataset[0].edge_features.shape[1],
    hidden_dim=64,
    out_dim=1,
)
mx.eval(model.parameters())

optimizer = optim.Adam(learning_rate=0.001)
loss_and_grad_fn = nn.value_and_grad(model, forward_fn)


In [19]:

epochs = 100
best_test_mse = 1e9
for e in range(epochs):
    loss, train_mse, test_mse = epoch()
    best_test_mse = min(best_test_mse, test_mse)

    print(
        " | ".join(
            [
                f"Epoch: {e:3d}",
                f"Train loss: {loss:.3f}",
                f"Train mse: {train_mse:.3f}",
                f"Test mse: {test_mse:.3f}",
            ]
        )
    )
print(f"\n==> Best test mse: {best_test_mse:.3f}")

Epoch:   0 | Train loss: 0.119 | Train mse: 2.813 | Test mse: 2.645
Epoch:   1 | Train loss: 0.050 | Train mse: 2.465 | Test mse: 2.499
Epoch:   2 | Train loss: 0.034 | Train mse: 1.838 | Test mse: 2.281
Epoch:   3 | Train loss: 0.029 | Train mse: 1.692 | Test mse: 1.969
Epoch:   4 | Train loss: 0.024 | Train mse: 1.315 | Test mse: 1.636
Epoch:   5 | Train loss: 0.023 | Train mse: 1.364 | Test mse: 1.574
Epoch:   6 | Train loss: 0.024 | Train mse: 1.138 | Test mse: 1.500
Epoch:   7 | Train loss: 0.019 | Train mse: 1.059 | Test mse: 1.356
Epoch:   8 | Train loss: 0.016 | Train mse: 0.905 | Test mse: 1.223
Epoch:   9 | Train loss: 0.017 | Train mse: 0.957 | Test mse: 1.122
Epoch:  10 | Train loss: 0.015 | Train mse: 0.813 | Test mse: 0.932
Epoch:  11 | Train loss: 0.013 | Train mse: 0.757 | Test mse: 0.983
Epoch:  12 | Train loss: 0.013 | Train mse: 0.727 | Test mse: 1.004
Epoch:  13 | Train loss: 0.014 | Train mse: 0.715 | Test mse: 0.842
Epoch:  14 | Train loss: 0.014 | Train mse: 0.83